# Analysis of AP systems

**usually: AA weak, AP medium, P dimerization, intra-factor $\alpha=10^5$ ($k^{\text{(a)}}_{on}=10^9$)**

## Preambles

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
#%matplotlib widget
import matplotlib as mpl
import matplotlib.backends.backend_pdf

In [3]:
#mpl.rcParams['figure.figsize'] = [8, 6]
#mpl.rcParams['figure.dpi'] = 300 #72

In [4]:
#
import kainit
import kamix
import kalarm
import kamol
import kaparam
import kareact
import kasig
import kasim
import kasystem
import kaheap
import kamatch
import kaviz
import kasnap
import kagraph
import kaplot

In [5]:
#import commands as cmd

import networkx as nx
import pandas as pd
import numpy as np
from math import ceil

import seaborn as sns

import matplotlib.pyplot as plt
import os
import bz2
import pickle
import sys

In [6]:
root = '/home/wf7/Walter/CombiFluids/PhaseDiagram/'
snapshot_directory = root + 'T25_A100_P20/'
csv_directory = snapshot_directory
csv_datafile = csv_directory + 'output_P_C20.csv'
results_directory = 'Analysis/'

In [123]:
print(f'snapshot_directory = {snapshot_directory}')
print(f'csv_directory = {csv_directory}')
print(f'csv_datafile = {csv_datafile}')
print(f'results_directory = {results_directory}')

snapshot_directory = /home/wf7/Walter/CombiFluids/PhaseDiagram/T25_A100_P20/
csv_directory = /home/wf7/Walter/CombiFluids/PhaseDiagram/T25_A100_P20/
csv_datafile = /home/wf7/Walter/CombiFluids/PhaseDiagram/T25_A100_P20/output_P_C20.csv
results_directory = Analysis/


## Quick simulation progress check


In [ ]:
data = pd.read_csv(csv_datafile, comment='#')
data.fillna(0, inplace=True)

In [ ]:
if 'pX' in locals(): plt.close(pX.fig)
pX = kaplot.XY_plot(params={'linestyle': '-', 'linewidth': 1, 'markersize': 2})
pX.ax.set_xscale('linear')
pX.ax.set_yscale('log')
pX.add(data, x='time', y='sz-rank 1', params={'label': 'max', 'color': 'r'})
pX.ax.legend();

## Acquire basic dataframes from snapshots

In [21]:
def patterns(snapshot, pats):

    # get a site graph matcher
    SGM = kamatch.SiteGraphMatcher()

    # turn the patterns into internal representations
    ip = [kamol.KappaComplex(p, canon=False) for p in pats]

    # read the snapshot
    print(f' scanning {snapshot}', end='\r')
    snap = kasnap.SnapShot(snapshot)
    print(f'analyzing {snapshot}', end='\r')

    all_lv = list(snap.local_views.keys())
    columns = ['complex', 'size', 'count', 'composition'] + all_lv + pats
    df = pd.DataFrame(columns=columns)
    data = []

    # sort complexes with decreasing size
    snap.complexes.sort(key=lambda m: m.size, reverse=True)

    idx = 1
    for c in snap.complexes:
        # distinguishable embeddings in c modulo automorphisms
        pattern_count = []
        local = []
        for p in ip:
            pattern_count.append(SGM.number_of_all_embeddings(c, p) / len(SGM.automorphisms(p)))
        for lv in all_lv:
            if lv in c.local_views:
                local.append(len(c.local_views[lv]))
            else:
                local.append(0)
        data.append([idx, c.size, c.count, c.composition] + local + pattern_count)
        idx += 1

    # fill the dataframe

    df = df.append(pd.DataFrame(data, columns=columns))
    print(f'done {" ":30}')
    return df

In [22]:
pats = ['A(p[.])', 
        'A(l[.])', 
        'A(r[.])', 
        'A(l[.] r[.])', 
        'P(d[.])', 'P(a1[.])', 
        'P(a2[.])',
        'P(a3[.])', 
        'A(r[1]), A(l[1])', 
        'A(p[1]), P(a1[1])', 
        'A(p[1]), P(a2[1])', 
        'A(p[1]), P(a3[1])', 
        'P(d[1]), P(d[1])']

In [23]:
df = patterns(snapshot_directory + 'snap154.ka', pats)

Exception: Cannot find snapshot file /home/wf7/Walter/CombiFluids/PhaseDiagram/T/snap154.ka

In [ ]:
df.set_index('complex', inplace=True)

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df

In [ ]:
n = 0
for i in range(1, len(df) + 1):
    n += df.loc[i,'patterns']['A(r[1]), A(l[1])'] * df.loc[i,'count']
print(f'[{i}] {n}')